el primer paso consiste en instalar las dependencias necesarias para poner en marcha el proyecto.

In [ ]:
%pip install langchain

In [ ]:
%pip install -qU langchain-openai


Luego, se configura una variable de entorno con la clave de API para conectar con OpenAI y se indica que se utilizará el modelo gpt-4 de ChatGPT.

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = ""

from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4")


Para verificar su funcionamiento, se crea una lista llamada messages que incluye dos mensajes:

SystemMessage: Instruye al modelo para traducir el texto siguiente del inglés al italiano.
HumanMessage: Contiene el mensaje "hi!", que es el texto que se desea traducir.

In [16]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from English into Italian"),
    HumanMessage(content="hi!"),
]

model.invoke(messages)

AIMessage(content='ciao!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 20, 'total_tokens': 23, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-e3e94e24-b1d3-4edb-a432-8491627b4d7a-0', usage_metadata={'input_tokens': 20, 'output_tokens': 3, 'total_tokens': 23, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})


En este paso, se toma la salida generada por el modelo y se convierte en una cadena de texto según la configuración establecida en StrOutputParser.

In [17]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

result = model.invoke(messages)

parser.invoke(result)

'Ciao!'


Luego, se utiliza el operador | para combinar el modelo y el parser, creando así una cadena (chain). En esta cadena, el modelo procesa los mensajes inicialmente y luego envía automáticamente el resultado al parser para su formateo.


In [18]:
chain = model | parser

chain.invoke(messages)

'ciao!'


En este paso, se crea una instancia de ChatPromptTemplate llamada prompt_template utilizando from_messages. Este método recibe una lista de mensajes, en la que cada elemento incluye un tipo de mensaje ("system" o "user") y una cadena con el contenido correspondiente.

El mensaje de tipo "system" emplea system_template, que instruye al modelo para traducir un texto a un idioma determinado. El mensaje de tipo "user" utiliza "{text}" como marcador de posición, el cual se rellenará con el contenido que el usuario desea traducir (en este caso, "hi").

In [19]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "Translate the following into {language}:"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

result = prompt_template.invoke({"language": "italian", "text": "hi"})

result

ChatPromptValue(messages=[SystemMessage(content='Translate the following into italian:', additional_kwargs={}, response_metadata={}), HumanMessage(content='hi', additional_kwargs={}, response_metadata={})])

La siguiente línea convierte el resultado en una lista de objetos, donde cada mensaje está en el formato adecuado para que el modelo comprenda la estructura de la conversación.

In [20]:
result.to_messages()

[SystemMessage(content='Translate the following into italian:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='hi', additional_kwargs={}, response_metadata={})]

Por último, se utiliza el operador | para unir prompt_template, model y parser en una cadena llamada chain. Esto permite que prompt_template procese los datos de entrada y cree un prompt formateado, el cual se envía automáticamente al model, que genera una respuesta. Finalmente, la salida de model se pasa al parser, que formatea o procesa la respuesta.

In [21]:
chain = prompt_template | model | parser

chain.invoke({"language": "italian", "text": "hi"})

'ciao'